# Лабораторная работа №5. Нормальное распределение и его приложения.

## Цель: исследовать распределение ошибок прогноза. Задачи: сравнить различные модели прогнозирования, проаналазировать нормальность распределения ошибок.

Генератор случайных чисел для генерации нормально распределенных данных.

In [ ]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import pandas as pd

class LMCG:
    def __init__(self, seed, a, m):
        self.state = seed
        self.a = a
        self.m = m

    def next(self):
        self.state = (self.a * self.state) % self.m
        return self.state/10000

    def generate_sequence(self, length):
        return [self.next() for _ in range(length)]

    def calculate_sequence_length(self):
        sequence = []  # Для хранения уникальных чисел
        index = 0       # Счётчик шагов
        while True:
            num = self.next()  # Генерация следующего числа
            if num in sequence:
                # Если число уже встречалось, возвращаем результаты
                first_occurrence_index = sequence.index(num)
                return (
                    len(sequence),  # Длина последовательности до повторения
                    num,            # Повторяющийся элемент
                    first_occurrence_index + 1,  # Номер первого появления (начиная с 1)
                    index + 1,      # Номер повторного появления (начиная с 1)
                )
            sequence.append(num)
            index += 1


def is_prime(n: int):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True


def get_prime_factors(n: int):
    factors = set()
    # Делим на 2
    while n % 2 == 0:
        factors.add(2)
        n //= 2
    # Делим на нечётные числа
    i = 3
    while i * i <= n:
        while n % i == 0:
            factors.add(i)
            n //= i
        i += 2
    if n > 2:
        factors.add(n)
    return sorted(factors)


def is_primitive_root(a, m, prime_factors):
    if pow(a, m - 1, m) != 1:  # Проверка малой теоремы Ферма
        return False
    for q in prime_factors:
        if pow(a, (m - 1) // q, m) == 1:
            return False
    return True


def find_all_primitive_roots(m, start=2, end=None):
    if not is_prime(m):
        raise ValueError("m должно быть простым числом.")

    if end is None:
        end = m - 1  # По умолчанию ищем до m-1

    # Находим простые делители m-1
    prime_factors = get_prime_factors(m - 1)

    primitive_roots = []
    for a in range(start, end + 1):
        if is_primitive_root(a, m, prime_factors):
            primitive_roots.append(a)
    return primitive_roots


def norm_generate(sequence, mean: float, std: float):
    eta = []
    # Берём числа попарно
    for i in range(0, len(sequence) - 1, 2):
        u1 = sequence[i]
        u2 = sequence[i + 1]
        # Метод Бокса-Мюллера
        z1 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
        z2 = np.sqrt(-2 * np.log(u1)) * np.sin(2 * np.pi * u2)
        eta.append(mean + std * z1)
        eta.append(mean + std * z2)
    return eta


Сгенерируем истинные значения данных по правилу: $y=3t+12+eps_t$, где $eps_t \sim N(0, 2^2)$

Введем две прогнозирующие модели:
$y_1 = 2.9t + 8$ и $y_2 = 3.6t + 4$

In [ ]:
t = np.array([(i+1) for i in range(100)])

seed = 1300
m = 8609  # Простое число
start = 2  # Начало диапазона
end = 4 * m  # Конец диапазона
roots = find_all_primitive_roots(m, start, end)
a = roots[1500]

generator = LMCG(seed=seed, a=a, m=m)
length = 100
sequence = generator.generate_sequence(length)

eps = np.array(norm_generate(sequence=sequence, mean=0, std=4))

y = 3 * t + 12 + eps

y_1 = 2.9 * t + 8
y_2 = 3.6 * t + 4

График истинных данных и прогнозирующих моделей
Найдем ошибки моделей и исследуем их распределение

In [ ]:

plt.figure()
plt.scatter(t, y, label="Истинные данные")
plt.plot(t, y_1, label=f"Модель: $y = 1.9x+11$", color='red')
plt.plot(t, y_2, label=f"Модель: $y = 2.1x+9$",color='magenta')
plt.title("Истинные данные и прогнозирующие модели")
plt.xlabel("t")
plt.ylabel("y")
plt.grid(True)
plt.show()

err_1 = y - y_1
err_2 = y - y_2
#Основные оценки ошибок
mean_1 = np.mean(err_1)
std_1 = np.std(err_1)

mean_2 = np.mean(err_2)
std_2 = np.std(err_2)

print(f"Среднее значение ошибок первой модели: {mean_1: .4f}")
print(f"Среднеквдратичное отклонение ошибок первой модели: {std_1: .4f}")

print(f"Среднее значение ошибок второй модели: {mean_2: .4f}")
print(f"Среднеквдратичное отклонение ошибок второй модели: {std_2: .4f}")

fig, axes = plt.subplots(1, 2, figsize=(10,4))

#Гистограммы полученных ошибок
k = 1.72 * (len(err_1) ** (1 / 3))
axes[0].hist(err_1, bins=int(k), color='skyblue', density=1, edgecolor='black')
axes[0].set_title("Гистограмма ошибок первой модели")
axes[0].set_xlabel("Ошибки")
axes[0].set_ylabel("Частота")

k = 1.72 * (len(err_2) ** (1 / 3))
axes[1].hist(err_2, bins=int(k), color='magenta', edgecolor='black', density=1)
axes[1].set_title("Гистограмма ошибок второй модели")
axes[1].set_xlabel("Ошибки")
axes[1].set_ylabel("Частота")

plt.tight_layout()
plt.show()


## результаты1

![title](https://raw.githubusercontent.com/Udav6/Mathem_model_2025/main/Fig_L5_1.png)

In [ ]:
Среднее значение ошибок первой модели:  8.1205
Среднеквдратичное отклонение ошибок первой модели:  4.7394
Среднее значение ошибок второй модели: -23.2295
Среднеквдратичное отклонение ошибок второй модели:  18.3169

![title](https://raw.githubusercontent.com/Udav6/Mathem_model_2025/main/Fig_L5_2.png)

Оценим распределение с помощью критерия Шапиро-Уилка

In [ ]:
alpha = 0.05
print(sps.shapiro(err_1))
if sps.shapiro(err_1)[1] >= alpha:
    print('Ошибки первой модели имеют нормальное распределение')
else:
    print('Ошибки первой модели не имеют нормальное распределение')

print(sps.shapiro(err_2))
if sps.shapiro(err_2)[1] >= alpha:
    print('Ошибки второй модели имеют нормальное распределение')
else:
    print('Ошибки второй модели не имеют нормальное распределение')


## результаты2

In [ ]:
ShapiroResult(statistic=np.float64(0.9837913585949399), pvalue=np.float64(0.2590253684738551))
Ошибки первой модели имеют нормальное распределение
ShapiroResult(statistic=np.float64(0.9584309296559954), pvalue=np.float64(0.003112930490766518))
Ошибки второй модели не имеют нормальное распределение

Оценим адекватность моделей
Для анализа адекватности моделей используем *критерий Дарьбина-Уотсона* и метрики $MAE$, $MSE$, а также коэффициент детерминации $R^2$.

In [ ]:
#Статстика Драбина-Уотсона
n = len(y)
DW_1 = sum(((y[i]-y_1[i]) - (y[i-1] - y_1[i-1]))**2 for i in range(1, n))/sum((y[i]-y_1[i])**2 for i in range(n))
DW_2 = sum(((y[i]-y_2[i]) - (y[i-1] - y_2[i-1]))**2 for i in range(1, n))/sum((y[i]-y_2[i])**2 for i in range(n))

print(f"Статистика Дарбина-Уотсона для первой модели: {DW_1: .4f}")
print(f"Статистика Дарбина-Уотсона для второй модели: {DW_2: .4f}")

def measure_of_inadequacy(y: list, y_pred: list):
    n = len(y)
    RSS = sum((y[i] - y_pred[i])**2 for i in range(n))
    MSE = RSS/n
    RMSE = np.sqrt(MSE)
    MAE = 1/n * sum(np.abs(y[i] - y_pred[i]) for i in range(n))
    return MAE, MSE, RMSE

MAE_1, MSE_1, RMSE_1 = measure_of_inadequacy(y, y_1)
MAE_2, MSE_2, RMSE_2 = measure_of_inadequacy(y, y_2)

print("Метрики первой модели")
print(f"MAE: {MAE_1: .4f}\nMSE: {MSE_1: .4f}\nRMSE: {RMSE_1: .4f}\n ")

print("Метрики второй модели")
print(f"MAE: {MAE_2: .4f}\nMSE: {MSE_2: .4f}\nRMSE: {RMSE_2: .4f}\n ")


## результаты3

In [ ]:
Статистика Дарбина-Уотсона для первой модели:  0.3820
Статистика Дарбина-Уотсона для второй модели:  0.0391

Метрики первой модели
MAE:  8.1568
MSE:  88.4038
RMSE:  9.4023
 
Метрики второй модели
MAE:  24.3774
MSE:  875.1218
RMSE:  29.5825

На основании среднего, далекого от нуля, можно сделать вывод о том, что ошибки имеют сильный сдвиг. Значение статистик для обеих моделей 
значительно меньше 2, а следовательно имеют высокую степень обратной автокорреляции. Метрики отклонения имеют высокое значение, следовательно модели 
плохо описывают данные. Таким образом, можно сделать вывод о низкой адекватности моделей.

In [ ]:
import scipy.stats as sps
eps = sps.norm(loc=0, scale=4).rvs(size=100)
print(eps)


## результаты4

In [ ]:
[ 5.6113991   1.33831841 -1.15111208  2.98239328 -8.42062373  1.52782244
 -0.6813122  -0.35066224 -4.91921645 -4.10117556  3.58085318 -0.20249759
  4.09545359  1.79940524  2.07245091  1.11959298  4.27545364  0.53466831
  1.64326097 -1.57243817 -3.86355208  4.22673621 -0.77077553 -1.05334326
  0.69925577 -0.37710121  1.44635113 -3.47135733 -5.84635726 -7.45874152
 -7.73459435  5.9718825  -4.12373853  0.66424778 -0.98104258 11.68651366
  1.17450362  4.15002662  8.37630849 -0.8561505  -4.44190786 -7.78163155
  0.36602063 -3.59571054  3.80684821  1.59971813 -1.11418676  2.7402572
 -4.99300259  5.80357085  1.4115041   1.27240328 -1.65286161  4.59971884
  2.76074688  7.35429802  3.85643953  0.70063928  6.6525806  -5.64057787
 -0.5854637  -2.17494731 -2.00911978  2.65528002 -1.1032156   0.31774717
 -2.15722967 -2.5615731  -1.86797016  3.34774242  8.30945742  1.38360563
  3.74813539  1.33266062 -0.47566311  4.18619073  3.77900433  3.13919725
  0.22430984  0.61800116 -4.80329421  1.67955886 -8.70617758 -2.904131
  1.90576043  3.79771218  4.63599573 -5.58433413  4.8920197   5.23843996
 -2.73423182  3.74159919 -2.70761804 -1.31448466  2.66841712  0.04187278
 -2.05935362  4.65061105 -0.84598006 -2.28324354]
